In [1]:
import re
from importlib import resources

import tiktoken

from loguru import logger

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import load_prompt
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory

import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

from bot import NewsBot
from bot import BotConfig


_config = BotConfig()

In [2]:
bot = NewsBot()

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
There was a problem when trying to write in your cache folder (/home/jovyan/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.
/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
bot.execute("Qual a última notícia de acidentes no bairro Dom Bosco?")



> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY
Human: Qual a última notícia de acidentes no bairro Dom Bosco?
AI: {"resposta": "nao sei"}


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:35:24.066 | DEBUG    | bot._newsbot:execute:206 - Pergunta original: Qual a última notícia de acidentes no bairro Dom Bosco?
2024-01-05 03:35:24.067 | DEBUG    | bot._newsbot:execute:207 - Pergunta melhorada: Qual a última notícia de acidentes no bairro Dom Bosco?



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconheca frases como "oi", "ola", "bom dia", "tudo bem:", etc.
- "Consulta de conteudo": Perguntas sobre noticias da base de dados, reconheca trechos como "quais sao as ultimas noticias", "o que foi reportado no bairro", etc.
- "": QUALQUER PERGUNTA QUE NAO SE ENCAIXE NAS CATEGORIAS ACIMA, ou que nao possa ser respondida com o contexto disponivel na base de dados, a qual contem noticias de Pocos de Caldas e regiao.

Escolha apenas uma das opcoes. Sua resposta deve conter APENAS a categoria.

## CHAT HISTORY
Human: Qual a última notícia de acidentes no bairro Dom Bosco?
AI: {"resposta": "nao sei"}


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:35:24.508 | DEBUG    | bot._newsbot:execute:210 - Intenção: Consulta de conteúdo



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
<data>2020-03-26</data>
<titulo>Poços está com 58 casos suspeitos de Covid-19; confira o último boletim</titulo>
<autor>ondapocos</autor>
<link>https://ondapocos.com.br/pocos-esta-com-58-casos-suspeitos-de-

2024-01-05 03:35:25.608 | INFO     | bot._newsbot:execute:228 - >>> Resposta: resposta: nao sei

link_noticia: 

data_noticia: 

titulo_noticia: 

autor_noticia: 





> Finished chain.


{'response': 'resposta: nao sei\n\nlink_noticia: \n\ndata_noticia: \n\ntitulo_noticia: \n\nautor_noticia: \n\n',
 'execution_id': 'e3d125de73fc49d9852e911668d82b12'}

# Functions

In [5]:
def get_prompt(key):

    logger.debug("Estou em 'get_prompt'")
    filepath = str(
        resources.files("bot.prompts").joinpath(f"{key}.json")
    )

    return load_prompt(filepath)

def self_get_chroma_collection():

    logger.debug("Estou em 'self_get_chroma_collection'")

    host_name = _config.VECTORDATABASE_HOSTNAME
    port = _config.VECTORDATABASE_PORT
    collection_name = _config.EMBEDDING_COLLECTION
    persist_directory = _config.VECTORDATABASE_PERSIST_DIRECTORY
    
    settings = Settings(allow_reset=True, anonymized_telemetry=True, persist_directory=persist_directory)
    chroma_client = chromadb.HttpClient(host=host_name, port = port, settings=settings)
    
    return chroma_client.get_collection(collection_name, embedding_function=self_embedder)


def self_format_history_message(messages, human_prefix: str = 'Human', ai_prefix: str = 'AI'):

    logger.debug("Estou em 'self_format_history_message'")
    for message in messages:
        if isinstance(message, langchain_core.messages.human.HumanMessage):
            yield f"{human_prefix}: {message.content}\n"
        
        elif isinstance(message, langchain_core.messages.ai.AIMessage):
            yield f"{ai_prefix}: {message.content}\n"

def self_get_standalone_question(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_get_standalone_question'")

    self_chain_standalone_question = LLMChain(
        llm=self_llm, prompt=self_prompt_standalone_question, verbose=self_verbose_chains
    )

    return self_chain_standalone_question.predict(history=kwargs["history"], human_input=message)

def self_get_user_intention(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_get_user_intention'")

    self_chain_intention = LLMChain(
        llm=self_llm, prompt=self_prompt_intention, verbose=self_verbose_chains
    )
    intention = self_chain_intention.predict(history=kwargs["history"], human_input=message)

    return intention.lower().replace("ú", "u").replace("í", "i")

def extract_dict_from_string(string):

    logger.debug("Estou em 'extract_dict_from_string'")

    match = re.search(r"{([^}]+)}", string)
    if match is None:
        return dict()

    json_content = match.group(1)

    try:
        json_dict = json.loads("{" + json_content + "}")
        return json_dict
    except json.JSONDecodeError as err:
        logger.error(str(err))
        return dict()

def self_predict(collection, query, n_neighbors: int = 1000, n_results: int = 10, **kwargs):

    logger.debug("Estou em 'self_predict'")

    def f(key, value, k):
        def limit(x):
            if isinstance(x, list):
                return x[:k]
            return x
        if isinstance(value, list):
            return (key, [limit(x) for x in value])
        return (key, None)
    
    
    res = collection.query(
        query_texts=query,
        n_results=n_neighbors,
        **kwargs
        # where={"metadata_field": "is_equal_to_this"},
        # where_document={"$contains":"search_string"}
    )
  
    return  dict([f(key, value, n_results)  for key, value in res.items()])

def self_count_tokens(context: str) -> int:

    logger.debug("Estou em 'self_count_tokens'")
    encoding = tiktoken.encoding_for_model(_config.LLM_MODEL_NAME)
    num_tokens = len(encoding.encode(context))
    
    return num_tokens

def self__set_query_content(documents, metadata):

    logger.debug("Estou em 'self__set_query_content'")
    context_list = [self__set_local_context(content, meta) for content, meta in zip(documents, metadata)]

    tokens = 4000
    max_tokens = 3600
    while tokens > max_tokens:
        tokens = self_count_tokens("\n".join(context_list))
        if tokens >= max_tokens:
            context_list.pop()

def self__set_local_context(content, meta):

    logger.debug("Estou em 'self__set_local_context'")
    return (
        f"<data>{meta['date']}</data>\n"
        f"<titulo>{meta['title']}</titulo>\n"
        f"<autor>{meta['author']}</autor>\n"
        f"<link>{meta['link']}</link>\n"
        f"<conteudo>{content}</conteudo>\n"
    )

def self_get_content(query, n_results=10, n_neighbors=1000):

    logger.debug("Estou em 'self_get_content'")

    result = self_predict(
        self_collection,
        query,
        n_results=n_results,
        n_neighbors=n_neighbors,
    )

    return self__set_query_content(result['documents'][0], result['metadatas'][0])

def self_handler_start_conversation(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_start_conversation'")

    self_chain_greeting = LLMChain(
        llm=self_llm_chat, prompt=self_prompt_greeting, verbose=self_verbose_chains, memory=kwargs["memory"]
    )

    return self_chain_greeting.predict(history=kwargs["history"], human_input=message)

def self_handler_query(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_query'")

    self_chain_query = LLMChain(
        llm=self_llm_chat, prompt=self_prompt_query, verbose=self_verbose_chains, memory=kwargs["memory"]
    )

    context = self_get_content(query=message)
    logger.debug(f"context = {context}")

    for i in range(3):
        response = self_chain_query.predict(history=kwargs["history"], human_input=message, context=context)
        resp_dict = extract_dict_from_string(response)
        if len(resp_dict) > 0:
            return (
                f'resposta: {resp_dict["resposta"]}\n\n'
                f'link_noticia: {resp_dict["link"]}\n\n'
                f'data_noticia: {resp_dict["data"]}\n\n'
                f'titulo_noticia: {resp_dict["titulo"]}\n\n'
                f'autor_noticia: {resp_dict["autor"]}\n\n'
            )
        continue
        
    return response
    

def self_handler_fallback(*args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_fallback'")

    return (
        "Desculpe, mas não posso responder a essa pergunta. "
        "Algo em que possa ajudar sobre notícias de Poços de Caldas e região?"
    )


# Atributes

In [6]:
self_llm = ChatOpenAI(model_name=_config.LLM_MODEL_NAME, temperature=0)
self_llm_chat = ChatOpenAI(model_name=_config.LLM_MODEL_NAME, temperature=0, model_kwargs={"stop": ["HUMAN_INPUT", "IA:"]})


self_embeddings = HuggingFaceEmbeddings(model_name=_config.HUGGINGFACE_EMBEDDING_MODEL_NAME)
self_embedder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=_config.HUGGINGFACE_EMBEDDING_MODEL_NAME)

self_verbose_chains = True

# Load utilitary prompts and chains
self_prompt_intention = get_prompt("prompt_user_intention")
self_prompt_standalone_question = get_prompt("prompt_standalone_question")

# Load chat prompts and chains
self_prompt_greeting = get_prompt("prompt_greeting")
self_prompt_query = get_prompt("prompt_query")

self_collection = self_get_chroma_collection()

self_memory = ConversationSummaryBufferMemory(
    llm=OpenAI(temperature=0),
    chat_history=ChatMessageHistory(),
    return_messages=True,
    memory_key="chat_history",
    input_key="human_input",
    human_prefix="Human",
    ai_prefix="AI"
)

/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
2024-01-05 03:36:36.902 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:36:36.903 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:36:36.904 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:36:36.905 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:36:36.905 | DEBUG    | __main__:self_get_chroma_collection:12 - Estou em 'self_get_chroma_collection'


In [46]:
self_collection

Collection(name=clips-mfaq)

In [57]:
self_collection.query(
    query_texts="Jardim Primavera",
    n_results=10,
    # **kwargs
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"}
)

{'ids': [['0d362403-caef-52a3-a5dd-f64ed58a76da',
   'd0e419c4-5490-568c-9624-61362bef5a06',
   'd3675c71-babe-540a-88aa-07f708544154',
   '9661e4cd-1986-5301-9a62-5ea85a0d5377',
   'daa15c4a-d9d5-5f64-b10d-145a22d5f6f6',
   'acc57c9a-eb90-503a-a46e-b812c8848f10',
   '477ce73a-a0c0-5324-91ca-c70e6df4bcce',
   '8a67bb4e-0408-5965-8985-5ebbb8163d55',
   '328bf19c-c26b-5065-be9a-a146d37b657d',
   'd99c27a6-19a2-52b3-bc98-b2166b52cee4']],
 'distances': [[20.5799617767334,
   22.137451171875,
   22.225343704223633,
   22.648582458496094,
   23.165489196777344,
   23.26180076599121,
   23.37055778503418,
   23.548870086669922,
   23.684303283691406,
   23.68735122680664]],
 'embeddings': None,
 'metadatas': [[{'author': 'ondapocos',
    'categories': 'Notícias',
    'date': '2020-09-21',
    'link': 'https://ondapocos.com.br/flores-de-jacaranda-mimoso-em-vias-e-parques-de-pocos-de-caldas-encantam/',
    'snippet': 'Por ruas e parques de Poços de Caldas, as flores de Jacarandá-mimoso estão da

# Execute

In [7]:
message = "Qual a última notícia de acidentes no bairro Dom Bosco?"

In [8]:
chat_history = "".join(self_format_history_message(self_memory.chat_memory.messages))
chat_history

2024-01-05 03:36:36.966 | DEBUG    | __main__:self_format_history_message:27 - Estou em 'self_format_history_message'


''

## Melhorar query

In [9]:
standalone_question = self_get_standalone_question(message=message, history=chat_history)
logger.debug(f"Pergunta original: {message}")
logger.debug(f"Pergunta melhorada: {standalone_question}")

2024-01-05 03:36:42.247 | DEBUG    | __main__:self_get_standalone_question:37 - Estou em 'self_get_standalone_question'




> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:36:43.245 | DEBUG    | __main__:<module>:2 - Pergunta original: Qual a última notícia de acidentes no bairro Dom Bosco?
2024-01-05 03:36:43.246 | DEBUG    | __main__:<module>:3 - Pergunta melhorada: Qual é a última notícia de acidentes no bairro Dom Bosco?



> Finished chain.


## Identificar intenção

In [10]:
intention = self_get_user_intention(message=standalone_question, history=chat_history)
logger.debug(f"Intenção: {intention}")

2024-01-05 03:36:52.422 | DEBUG    | __main__:self_get_user_intention:47 - Estou em 'self_get_user_intention'




> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconheca frases como "oi", "ola", "bom dia", "tudo bem:", etc.
- "Consulta de conteudo": Perguntas sobre noticias da base de dados, reconheca trechos como "quais sao as ultimas noticias", "o que foi reportado no bairro", etc.
- "": QUALQUER PERGUNTA QUE NAO SE ENCAIXE NAS CATEGORIAS ACIMA, ou que nao possa ser respondida com o contexto disponivel na base de dados, a qual contem noticias de Pocos de Caldas e regiao.

Escolha apenas uma das opcoes. Sua resposta deve conter APENAS a categoria.

## CHAT HISTORY


## HUMAN INPUT
Qual é a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:36:52.964 | DEBUG    | __main__:<module>:2 - Intenção: consulta de conteudo



> Finished chain.


## Handlers

In [ ]:
def self_handler_query(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_query'")

    self_chain_query = LLMChain(
        llm=self_llm_chat, prompt=self_prompt_query, verbose=self_verbose_chains, memory=kwargs["memory"]
    )

    context = self_get_content(query=message)
    logger.debug(f"context = {context}")

    for i in range(3):
        response = self_chain_query.predict(history=kwargs["history"], human_input=message, context=context)
        resp_dict = extract_dict_from_string(response)
        if len(resp_dict) > 0:
            return (
                f'resposta: {resp_dict["resposta"]}\n\n'
                f'link_noticia: {resp_dict["link"]}\n\n'
                f'data_noticia: {resp_dict["data"]}\n\n'
                f'titulo_noticia: {resp_dict["titulo"]}\n\n'
                f'autor_noticia: {resp_dict["autor"]}\n\n'
            )
        continue
        
    return response

In [18]:
def self__set_query_content(documents, metadata):

    logger.debug("Estou em 'self__set_query_content'")
    context_list = [self__set_local_context(content, meta) for content, meta in zip(documents, metadata)]

    tokens = 4000
    max_tokens = 3600
    while tokens > max_tokens:
        tokens = self_count_tokens("\n".join(context_list))
        if tokens >= max_tokens:
            context_list.pop()

def self__set_local_context(content, meta):

    logger.debug("Estou em 'self__set_local_context'")
    return (
        f"<data>{meta['date']}</data>\n"
        f"<titulo>{meta['title']}</titulo>\n"
        f"<autor>{meta['author']}</autor>\n"
        f"<link>{meta['link']}</link>\n"
        f"<conteudo>{content}</conteudo>\n"
    )

def self_get_content(query, n_results=10, n_neighbors=1000):

    logger.debug("Estou em 'self_get_content'")

    result = self_predict(
        self_collection,
        query,
        n_results=n_results,
        n_neighbors=n_neighbors,
    )


    # logger.debug(f"result = {result}")
    return self__set_query_content(result['documents'][0], result['metadatas'][0])


In [ ]:
self_chain_query = LLMChain(
    llm=self_llm_chat, prompt=self_prompt_query, verbose=self_verbose_chains, memory=self_memory
)

# context = self_get_content(query=message)


result = self_predict(
    self_collection,
    query=message,
    n_results=10,
    n_neighbors=1000,
)

In [28]:
message

'Qual a última notícia de acidentes no bairro Dom Bosco?'

In [31]:
def self_predict(collection, query, n_neighbors: int = 1000, n_results: int = 10, **kwargs):

    logger.debug("Estou em 'self_predict'")

    def f(key, value, k):
        def limit(x):
            if isinstance(x, list):
                return x[:k]
            return x
        if isinstance(value, list):
            return (key, [limit(x) for x in value])
        return (key, None)
    
    
    res = collection.query(
        query_texts=query,
        n_results=n_neighbors,
        **kwargs
        # where={"metadata_field": "is_equal_to_this"},
        # where_document={"$contains":"search_string"}
    )
  
    return  dict([f(key, value, n_results)  for key, value in res.items()])

In [38]:
message

'Qual a última notícia de acidentes no bairro Dom Bosco?'

In [32]:
collection

NameError: name 'collection' is not defined

In [30]:



[x for x in result['documents'][0]]

# logger.debug(f"context = {context}")

['Segundo Boletim atualizado pela Secretaria Municipal de Saúde, neste domingo, 10 de maio, são 702 notificações em Poços. Destas, 430 se referem a investigações concluídas, sendo 38 descartadas por exame laboratorial e 392 com alta por critério clínico. São 245 casos suspeitos em investigação, dos quais: 241 seguem em isolamento domiciliar, casos leves sem indicação de realizar exame, conforme o Ministério da Saúde. Três pacientes estão hospitalizados na Unidade de Terapia Intensiva e um óbito é investigado. Neste domingo são 27 casos positivos em Poços: 21 pacientes são considerados recuperados, dois seguem em isolamento domiciliar, uma pessoa está hospitalizada e três pacientes vieram a óbito. Informe Estadual Segundo o Informe Epidemiológico divulgado pela Secretaria de Estado da Saúde de Minas Gerais, apresentado no site\xa0www.saude.mg.gov.br/coronavirus\xa0– “Distribuição dos casos de COVID-19 em Minas Gerais”, neste domingo,10, no Estado são 112.830 notificações; 99.958 casos s

In [23]:
len(result['metadatas'])

1

In [11]:
handlers = {
    "inicio de conversa": self_handler_start_conversation,
    "consulta de conteudo": self_handler_query,
    "": self_handler_fallback
}


response = handlers[intention](
    message=message,
    memory=self_memory,
    history=chat_history
)

logger.debug(f"Response: {response}")

2024-01-05 03:37:04.869 | DEBUG    | __main__:self_handler_query:153 - Estou em 'self_handler_query'
2024-01-05 03:37:04.870 | DEBUG    | __main__:self_get_content:130 - Estou em 'self_get_content'
2024-01-05 03:37:04.871 | DEBUG    | __main__:self_predict:75 - Estou em 'self_predict'
2024-01-05 03:37:05.273 | DEBUG    | __main__:self__set_query_content:107 - Estou em 'self__set_query_content'
2024-01-05 03:37:05.274 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:37:05.274 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:37:05.275 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:37:05.276 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:37:05.276 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:37:05.277 | DEBUG    | __main__:se



> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:37:05.978 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:37:06.407 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:37:06.914 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'
2024-01-05 03:37:06.914 | DEBUG    | __main__:<module>:14 - Response: Não sei.



> Finished chain.
